In [7]:
import math
import numpy as np
import scipy as sp
np.set_printoptions(threshold = 10001)

# Степенной метод

In [8]:
def power_iteration1(A, eps: float):
    b_k = np.random.rand(A.shape[1])
    while True:
        t = np.dot(A, b_k)
        b_k0 = b_k
        b_k = t / np.linalg.norm(t)
        lambda_k0 = np.dot(b_k0.T, t) / np.dot(b_k0.T, b_k0)
        lambda_k = np.dot(b_k.T, np.dot(A, b_k)) / np.dot(b_k.T, b_k)
        if ((abs(lambda_k - lambda_k0) / abs(lambda_k0)) < eps):
            break
    return abs(lambda_k)

# Функция для поиска оптимального параметра

Функция получает на вход матрицу A, столбец b, размеры матрицы N и начальное приближение $\tau_0$. Мы знаем, что зона поиска - интервал $(0, \frac{2}{\lambda_{max}})$, поэтому делаем следующее: мы вокруг $\tau_0$, которое и есть $\frac{2}{\lambda_{max}}$, берем две точки $\tau_1$, $\tau_2$ и теперь для каждой из трех $\tau$ считаем 10 итераций метода Ричардсона. После чего сравниваем нормы невязок в каждом случае и запускаемся снова с тем $\tau$, для которого норма невязки наименьшая(не забывая проверить, что максимальное по модулю собственное число меньше единицы).
Критерием остановки тут служит малое изменение нормы невязки, т.е. если норма невязки меняется меньше, чем на $\epsilon = 1$, то мы останавливаемся с текущим $\tau$.

In [9]:
def tau(A: list, b: list, N: int, tau_opt0: float) -> float:
    tau_opt1 = tau_opt0 + (1 / N)
    tau_opt2 = tau_opt0 - (1 / N)
    x = np.zeros(N + 1)
    for i in range(10):
        nev0 = b - (A @ x)
        x = x + (tau_opt0 * nev0)
    x = np.zeros(N + 1)
    for k in range(10):
        nev1 = b - (A @ x)
        x = x + (tau_opt1 * nev1)
    x = np.zeros(N + 1)
    for t in range(10):
        nev2 = b - (A @ x)
        x = x + (tau_opt2 * nev2)
    T = np.eye(N + 1)
    if (np.linalg.norm(nev0) - np.linalg.norm(nev1) >= 1)and(power_iteration1(T - tau_opt1*A, 10**(-5)) < 1):
        if (np.linalg.norm(nev1) - np.linalg.norm(nev2) >= 1)and(power_iteration1(T - tau_opt2*A, 10**(-5)) < 1):
            return tau(A, b, N, tau_opt2)
        return tau(A, b, N, tau_opt1)
    if (np.linalg.norm(nev0) - np.linalg.norm(nev2) >= 1)and(power_iteration1(T - tau_opt2*A, 10**(-5)) < 1):
        return tau(A, b, N, tau_opt2)
    return tau_opt0

# Метод Ричардсона

In [10]:
def Rich_for_A(A: list, b: list, eps: float) -> list:
    C = A.T @ A
    B = A.T @ b
    N = len(A) - 1
    lambda_max = power_iteration1(C, eps)
    T = np.eye(N + 1)
    lambda_min = power_iteration1(C, eps)-power_iteration1(C - lambda_max*T, eps)
    mu = (lambda_max / lambda_min)**(1/2)
    lambda_max = power_iteration1(A, eps)
    x = np.zeros(N + 1)
    tau_opt = tau(A, b, N, (2 / lambda_max))
    while True:
        nev = b - (A @ x)
        x = x + (tau_opt * nev)
        nev1 = b - (A @ x)
        if (np.linalg.norm(nev) < np.linalg.norm(nev1)):
            break
        if ((mu * np.linalg.norm(nev) / np.linalg.norm(b)) < eps):
            break
    return x

# Тест на матрице из дз

In [21]:
N = 99
i = 0
k = 0
A = np.zeros((N + 1, N + 1))
for i in range(N + 1):
    if i < 5:
        for k in range(i + 5):
            A[i][k] = 1
        A[i][i] = 10
        continue
    if i >= N - 4:
        for k in range(5 + (N - i)):
            A[i][N-k] = 1
        A[i][i] = 10
        continue
    A[i][i - 4] = 1
    A[i][i - 3] = 1
    A[i][i - 2] = 1
    A[i][i - 1] = 1
    A[i][i] = 10
    A[i][i + 1] = 1
    A[i][i + 2] = 1
    A[i][i + 3] = 1
    A[i][i + 4] = 1
b = np.zeros(N + 1)
for i in range(N + 1):
    b[i] = i + 1
eps = 10**(-5)
print("Вектор ответов имеет вид: " + str(Rich_for_A(A, b, eps)))

Вектор ответов имеет вид: [0.0229216  0.09689632 0.16487606 0.22670647 0.28230881 0.33422872
 0.38818381 0.44352799 0.49958075 0.55564397 0.61130217 0.66678018
 0.72223329 0.77774151 0.83330658 0.8888834  0.94445169 1.00000851
 1.05555999 1.11111247 1.16666732 1.22222358 1.27777999 1.33333593
 1.38889152 1.44444702 1.50000258 1.55555823 1.61111391 1.66666958
 1.72222524 1.77778089 1.83333654 1.88889219 1.94444784 2.00000349
 2.05555915 2.1111148  2.16667045 2.2222261  2.27778176 2.33333741
 2.38889306 2.44444871 2.50000437 2.55556002 2.61111567 2.66667132
 2.72222698 2.77778263 2.83333828 2.88889393 2.94444959 3.00000524
 3.05556089 3.11111654 3.16667219 3.22222785 3.2777835  3.33333915
 3.38889479 3.44445042 3.50000607 3.55556174 3.61111746 3.66667314
 3.72222867 3.77778398 3.83333927 3.88889511 3.94445208 4.00000963
 4.0555654  4.11111613 4.1666615  4.22221152 4.27778272 4.33338314
 4.38898723 4.44450225 4.49986894 4.55516115 4.61067833 4.66702509
 4.72391005 4.78015151 4.83372689 4.